In [8]:
import pandas as pd
from glob import glob

In [9]:
datasets = ["agnews", "sst-5", "gsm8k", "copa", "subj"]
models = ["llama", "qwen", "mistral"]
seeds = [42, 43, 44]
dfs = []
for dataset in datasets:
    for model in models:
        for seed in seeds:
            path = glob(
                f"../results/main_results/{dataset}/{model}/CAPO/seed{seed}/*/*/prompt_scores.parquet"
            )[0]
            df = pd.read_parquet(path)
            df["evaluated_blocks"] = 10 - df.isnull().sum(axis=1)
            df["evaluated_blocks_when_no_racing"] = 10
            dfs.append(
                df.assign(
                    dataset=dataset,
                    model=model,
                    seed=seed,
                )
            )

df = pd.concat(dfs, ignore_index=True)

In [10]:
df = df.groupby(["dataset", "model", "seed"]).agg(
    mean_evaluated_blocks=("evaluated_blocks", "sum"),
    mean_evaluated_blocks_when_no_racing=("evaluated_blocks_when_no_racing", "sum"),
)

# build mean
df = df.groupby(["dataset", "model"]).agg(
    mean_evaluated_blocks=("mean_evaluated_blocks", "mean"),
    mean_evaluated_blocks_when_no_racing=("mean_evaluated_blocks_when_no_racing", "mean"),
)

In [11]:
df["saved_evals"] = df["mean_evaluated_blocks_when_no_racing"] - df["mean_evaluated_blocks"]
df["saved_evals_ratio"] = df["saved_evals"] / df["mean_evaluated_blocks_when_no_racing"]

In [12]:
df

mean_evaluated_blocks  mean_evaluated_blocks_when_no_racing  \
dataset model                                                                  
agnews  llama               929.000000                           1886.666667   
        mistral             608.333333                           1356.666667   
        qwen                707.000000                           1310.000000   
copa    llama               804.666667                           1690.000000   
        mistral             754.666667                           1273.333333   
        qwen                948.666667                           1566.666667   
gsm8k   llama               317.666667                            630.000000   
        mistral             314.000000                            456.666667   
        qwen                376.666667                            633.333333   
sst-5   llama               832.666667                           1316.666667   
        mistral             703.333333                           1093.333333   
        qwen                836.333333                           1070.000000   
subj    llama               648.333333                           1566.666667   
        mistral             625.000000                           1260.000000   
        qwen                672.666667                           1360.000000   

                 saved_evals  saved_evals_ratio  
dataset model                                    
agnews  llama     957.666667           0.507597  
        mistral   748.333333           0.551597  
        qwen      603.000000           0.460305  
copa    llama     885.333333           0.523866  
        mistral   518.666667           0.407330  
        qwen      618.000000           0.394468  
gsm8k   llama     312.333333           0.495767  
        mistral   142.666667           0.312409  
        qwen      256.666667           0.405263  
sst-5   llama     484.000000           0.367595  
        mistral   390.000000           0.356707  
        qwen      233.666667           0.218380  
subj    llama     918.333333           0.586170  
        mistral   635.000000           0.503968  
        qwen      687.333333           0.505392

In [13]:
df.mean()

mean_evaluated_blocks                    671.933333
mean_evaluated_blocks_when_no_racing    1231.333333
saved_evals                              559.400000
saved_evals_ratio                          0.439788
dtype: float64